In [11]:
!pip install -q evaluate peft

In [ ]:
!pip uninstall -y backports

In [12]:
!python --version

Python 3.11.7


In [13]:
import argparse
import os
import peft

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    LoHaConfig,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
)

import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup, set_seed
from tqdm import tqdm

ImportError: cannot import name 'tarfile' from 'backports' (/home/chinju/anaconda3/lib/python3.11/site-packages/backports/__init__.py)

In [ ]:
batch_size = 1
model_name_or_path = "Rostlab/prot_bert"
#task = "mrpc"
#peft_type = PeftType.LoHa
device = "cuda"
num_epochs = 5

In [ ]:
peft_config = LoHaConfig(task_type="SEQ_CLS", inference_mode=False, r=16, alpha=16, module_dropout=0.1,target_modules=["query","value"])
lr = 3e-4

NameError: name 'LoHaConfig' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

#BENCHMARKS_DIR=  #give path to your data

BENCHMARKS_DIR = 'PATH to your data/input'
BENCHMARK_NAME = 'input'

In [ ]:
import os
import pandas as pd
train_set_file_path = os.path.join(BENCHMARKS_DIR,'%s.train.csv' % BENCHMARK_NAME)
my_train=pd.read_csv(train_set_file_path)
test_set_file_path = os.path.join(BENCHMARKS_DIR,'%s.test.csv' % BENCHMARK_NAME)
my_test=pd.read_csv(test_set_file_path)

In [ ]:
from sklearn.model_selection import train_test_split
lab = []
for i in my_train['flabel']:
  try:
    lab.append(int(i))
  except:
    lab.append(0)

In [ ]:
my_train['label']=lab
my_train, my_valid = train_test_split(my_train, stratify = my_train['label'], test_size = 0.1, random_state = 0)
print(f'{len(my_train)} training set records, {len(my_valid)} validation set records, {len(my_test)} test set records.')

6395 training set records, 711 validation set records, 832 test set records.


In [ ]:
my_train.rename(columns = {'seq':'sequence'}, inplace = True)
my_valid.rename(columns = {'seq':'sequence'}, inplace = True)
#my_train.rename(columns = {'flabel':'label'}, inplace = True)
my_test.rename(columns = {'seq':'sequence'}, inplace = True)
my_test.rename(columns = {'flabel':'label'}, inplace = True)

In [ ]:
# Dataset creation
from datasets import Dataset
def create_dataset(tokenizer,seqs,labels):
    tokenized = tokenizer(seqs, max_length=1024, padding=True, truncation=True)
    dataset = Dataset.from_dict(tokenized)
    #datas=Dataset.from_dict
    dataset = dataset.add_column("labels", labels)

    return dataset

In [ ]:
 # Preprocess inputs
    # Replace uncommon AAs with "X"
my_train["sequence"]=my_train["sequence"].str.replace('|'.join(["O","B","U","Z"]),"X",regex=True)
my_valid["sequence"]=my_valid["sequence"].str.replace('|'.join(["O","B","U","Z"]),"X",regex=True)
    # Add spaces between each amino acid for PT5 to correctly use them
my_train['sequence']=my_train.apply(lambda row : " ".join(row["sequence"]), axis = 1)
my_valid['sequence']=my_valid.apply(lambda row : " ".join(row["sequence"]), axis = 1)

In [ ]:
#from transformers import T5EncoderModel, T5Tokenizer,AutoModelForSequenceClassification
from transformers import BertForMaskedLM, BertTokenizer,AutoModelForSequenceClassification
padding_side = "right"

tokenizer = BertTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)

/home/chinju/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")

In [ ]:
train_set=create_dataset(tokenizer,list(my_train['sequence']),my_train['label'])
valid_set=create_dataset(tokenizer,list(my_valid['sequence']),my_valid['label'])

In [ ]:
#del train_dataloader,eval_dataloader

In [ ]:
train_dataloader = DataLoader(train_set, shuffle=True,collate_fn=collate_fn,batch_size=batch_size)
eval_dataloader = DataLoader(valid_set, shuffle=False,collate_fn=collate_fn, batch_size=batch_size)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True,num_labels=5)
model = get_peft_model(model, peft_config)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Rostlab/prot_bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoHaModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30, 1024, padding_idx=0)
          (position_embeddings): Embedding(40000, 1024)
          (token_type_embeddings): Embedding(2, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-29): 30 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): loha.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (hada_w1_a): ParameterDict(  (default): Parameter containing: [torch.FloatTensor of size 1024x16])
                    (hada_w1_b): ParameterDict(  (default): Parameter containing: [to

In [ ]:
model.print_trainable_parameters()

trainable params: 3,937,285 || all params: 423,873,546 || trainable%: 0.9289


In [ ]:
optimizer = AdamW(params=model.parameters(), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
#metric1 = evaluate.load("f1",average='macro')
metric = evaluate.load("accuracy")

In [ ]:

model.to(device)
for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )
        #metric1.add_batch(
           # predictions=predictions,
            #references=references,
            #average="macro"

        #)

    eval_metric = metric.compute()
    #f1_score=metric1.compute()
    print(f"epoch {epoch}:", eval_metric)

100%|█████████████████████████████████████████| 711/711 [02:25<00:00,  4.87it/s]


epoch 0: {'accuracy': 0.9226441631504922}


100%|█████████████████████████████████████████| 711/711 [02:26<00:00,  4.86it/s]


epoch 1: {'accuracy': 0.9310829817158931}


100%|█████████████████████████████████████████| 711/711 [02:26<00:00,  4.87it/s]


epoch 2: {'accuracy': 0.9831223628691983}


100%|█████████████████████████████████████████| 711/711 [02:26<00:00,  4.86it/s]


epoch 3: {'accuracy': 0.9831223628691983}


100%|█████████████████████████████████████████| 711/711 [02:25<00:00,  4.87it/s]

epoch 4: {'accuracy': 0.9831223628691983}


## Share adapters on the 🤗 Hub

In [ ]:
!huggingface-cli login --token "token_here"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/chinju/.cache/huggingface/token
Login successful


In [ ]:
model.push_to_hub("huggingface_account", use_auth_token=True)

/home/chinju/anaconda3/lib/python3.11/site-packages/transformers/utils/hub.py:834: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/chinju/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Chinjuj/protebert-protfam-peft-loha/commit/0b7433184d4fdc5552df0f198712793a58fba500', commit_message='Upload model', commit_description='', oid='0b7433184d4fdc5552df0f198712793a58fba500', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

You can also directly load adapters from the Hub using the commands below:

In [ ]:
my_test["sequence"]=my_test["sequence"].str.replace('|'.join(["O","B","U","Z"]),"X",regex=True)
my_test['sequence']=my_test.apply(lambda row : " ".join(row["sequence"]), axis = 1)

In [ ]:
test_set=create_dataset(tokenizer,list(my_test['sequence']),my_test['label'])
eval_dataloader = DataLoader(test_set, shuffle=False,collate_fn=collate_fn, batch_size=batch_size)

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer,BertTokenizer,AutoModelForSequenceClassification

peft_model_id = "Chinjuj/protebert-protfam-peft-loha"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path,num_labels=5)
tokenizer = BertTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
inference_model = PeftModel.from_pretrained(inference_model, peft_model_id)

inference_model.to(device)
inference_model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = inference_model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    metric.add_batch(
        predictions=predictions,
        references=references,
    )

eval_metric = metric.compute()
print(eval_metric)

adapter_config.json:   0%|          | 0.00/514 [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Rostlab/prot_bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

100%|█████████████████████████████████████████| 832/832 [02:45<00:00,  5.04it/s]


{'accuracy': 0.9819711538461539}
